# 서로 다른 모델과 분할된 데이터, 그 결과에 대한 병합
---

## 개요
승순님이 만들어주신 100개의 chunk파일 중 일부를 사용하여 모델에 바로 입력할 수 있도록 데이터 파이프라인을 생성합니다.  
데이터 파이프라인을 통과한 chunk file들로 CNN, DenseNet, ConvMixer 모델을 학습시킵니다. 각각의 모델을 사용하는 이유는 다음과 같습니다.  
* **CNN**: CNN 모델을 통해 이미지의 국소적인 특징을 추출하기 위함입니다.
* **DenseNet**: DenseNet 모델을 통해 이미지의 전체적인 특징을 추출하기 위함입니다.
* **ConvMixer**: ConvMixer 모델을 통해 이미지와 토큰의 관계에 대한 보다 정확한 특징을 추출하기 위함입니다.  

생성된 모델들은 아키텍처가 다르기 때문에 Weighted Average 앙상블을 진행하여 각 모델의 예측에 가중치를 부여하여 각각의 결과를 결합합니다.  
일련의 과정들을 모듈화하여 각기 다른 chunk file로 학습을 진행하여 그 결과를 비교합니다.

## 목차
* 데이터 압축해제 및 확인하기
* 파이프라인 구축하기
* 모델 구조 설계하기
* 각각의 모델 학습하기
* 학습 결과 병합하기

### STEP 1. 데이터 압축해제 및 확인하기
---
압축된 상태의 청크 파일을 압축해제하고, 파일이 어떻게 구성되어 있는지 확인합니다.

In [1]:
import os

# simple 압축 파일 경로 지정
compressed_file_path = os.getenv('HOME') + '/aiffel/Kaggle_Hackathon/data/simple_shuffle_data.zip'
# 압축 해제를 진행할 경로 지정
gz_file_path = os.getenv('HOME') + '/aiffel/Kaggle_Hackathon/data/compressed_simple'

In [2]:
import zipfile

# .zip 압축파일을 지정된 경로에 .gz형태의 chunk file들로 압축해제
def extract_zip(current_file_path, extract_file_path):
    with zipfile.ZipFile(current_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_file_path)
        

# 압축 해제 진행
extract_zip(compressed_file_path, gz_file_path)

# 정상적으로 압축해제가 진행됐는지 확인
print(os.listdir(gz_file_path))

['train_k11.gz', 'train_k49.gz', 'train_k62.gz', 'train_k29.gz', 'train_k66.gz', 'train_k24.gz', 'train_k61.gz', 'train_k32.gz', 'train_k44.gz', 'train_k96.gz', 'train_k70.gz', 'train_k30.gz', 'train_k63.gz', 'train_k14.gz', 'train_k57.gz', 'train_k1.gz', 'train_k6.gz', 'train_k19.gz', 'train_k40.gz', 'train_k7.gz', 'train_k42.gz', 'train_k37.gz', 'train_k51.gz', 'train_k91.gz', 'train_k4.gz', 'train_k72.gz', 'train_k95.gz', 'train_k90.gz', 'train_k85.gz', 'train_k21.gz', 'train_k35.gz', 'train_k50.gz', 'train_k5.gz', 'train_k8.gz', 'train_k2.gz', 'train_k98.gz', 'train_k94.gz', 'train_k99.gz', 'train_k39.gz', 'train_k23.gz', 'train_k81.gz', 'train_k83.gz', 'train_k55.gz', 'train_k84.gz', 'train_k59.gz', 'train_k86.gz', 'train_k28.gz', 'train_k31.gz', 'train_k87.gz', 'train_k89.gz', 'train_k97.gz', 'train_k80.gz', 'train_k18.gz', 'train_k92.gz', 'train_k20.gz', 'train_k69.gz', 'train_k47.gz', 'train_k73.gz', 'train_k79.gz', 'train_k53.gz', 'train_k33.gz', 'train_k3.gz', 'train_k58.gz',

In [4]:
import gzip
import shutil

# .gz 형태의 압축파일을 지정된 경로와 이름으로 압축해제
def extract_gz(extract_file_path):
    for i in range(100):
        compressed_file_path = os.path.join(gz_file_path,f'train_k{i}.gz')
        file_name = os.path.basename(compressed_file_path)
        extract_path = os.path.join(extract_file_path, os.path.splitext(file_name)[0])

        with gzip.open(compressed_file_path, 'rb') as f_in:
            with open(extract_path, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)


# 압축해제를 진행한 후 생성되는 파일들의 저장 경로 지정                
file_path = os.getenv('HOME') + '/aiffel/Kaggle_Hackathon/data/simple'
# 압축해제 진행
extract_gz(file_path)

# 정상적으로 압축해제가 진행됐는지 확인
print(os.listdir(file_path))

['train_k6', 'train_k41', 'train_k55', 'train_k28', 'train_k78', 'train_k9', 'train_k43', 'train_k20', 'train_k61', 'train_k98', 'train_k18', 'train_k62', 'train_k27', 'train_k63', 'train_k38', 'train_k44', 'train_k16', 'train_k56', 'train_k97', 'train_k99', 'train_k33', 'train_k40', 'train_k60', 'train_k3', 'train_k47', 'train_k73', 'train_k49', 'train_k21', 'train_k12', 'train_k80', 'train_k66', 'train_k25', 'train_k74', 'train_k30', 'train_k71', 'train_k48', 'train_k11', 'train_k95', 'train_k65', 'train_k37', 'train_k31', 'train_k7', 'train_k2', 'train_k26', 'train_k8', 'train_k45', 'train_k83', 'train_k4', 'train_k94', 'train_k50', 'train_k77', 'train_k70', 'train_k86', 'train_k57', 'train_k29', 'train_k90', 'train_k93', 'train_k79', 'train_k67', 'train_k39', 'train_k32', 'train_k58', 'train_k92', 'train_k34', 'train_k68', 'train_k64', 'train_k36', 'train_k53', 'train_k10', 'train_k88', 'train_k51', 'train_k1', 'train_k59', 'train_k5', 'train_k76', 'train_k87', 'train_k23', 'train_

In [5]:
import pandas as pd
import os

# 0번 인덱스의 chunk file 경로 지정
file_path = os.getenv('HOME') + '/aiffel/Kaggle_Hackathon/data/simple/train_k0'
# chunk file 읽기
data_df = pd.read_csv(file_path)
# chunk file 데이터 확인
print(data_df.head())

                                             drawing    y
0  [[[3, 4, 1, 114, 208, 248, 255, 157, 90, 28, 1...  194
1  [[[50, 44, 29, 9, 0, 5, 25], [57, 55, 58, 70, ...  198
2  [[[133, 156, 161, 167, 190, 204, 228, 251, 255...   16
3  [[[0, 9, 25, 6, 1, 3, 10, 19, 35, 57, 89, 115,...  221
4  [[[160, 246, 253, 254, 227, 190, 177, 167, 149...  173


### STEP 2. 파이프라인 구축하기
---
압축해제된 데이터들 중 학습에 사용할 데이터를 핸들링하여 모델에 바로 입력할 수 있도록 준비합니다.

In [ ]:
import os
import pandas as pd
import ast
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.inception_v3 import preprocess_input
import pickle

# 데이터 파이프라인 함수 정의
def data_pipeline(dir_path, pkl_path, file_idx):
    # 데이터와 레이블을 담을 리스트 생성
    data_list = []
    label_list = []

    # 데이터 읽기
    data = pd.read_csv(os.path.join(dir_path, f'train_k{file_idx}'))
    # drawing 열을 리스트로 변환
    drawing = data['drawing'].apply(ast.literal_eval).tolist()
    
    # 레이블 이름이 담긴 파일 읽기
    with open(pkl_path, 'rb') as f:
        label_names = pickle.load(f)

    for draw_idx, label_idx in zip(drawing, data['y']):
        # 256X256 크기의 흰색 이미지 생성
        img = Image.new('L', (256, 256), color='white')
        # 배경으로 지정
        draw = ImageDraw.Draw(img)

        # 배열의 각 요소를 만들어 놓은 이미지에 그리기
        for stroke in draw_idx:
            for i in range(len(stroke[0]) - 1):
                draw.line([stroke[0][i], stroke[1][i], stroke[0][i+1], stroke[1][i+1]], fill='black', width=3)

        # 메모리 절감을 위해 이미지의 크기를 28X28로 변환
        img = img.resize((28, 28))

        # 이미지를 배열의 형태로 변환
        image_array = img_to_array(img)
        # 변환된 데이터를 모델에 입력하기 용이하도록 텐서로 변환
        image_array = preprocess_input(image_array)

        # 학습 데이터 리스트에 추가
        data_list.append(tf.convert_to_tensor(image_array, dtype=tf.float32))
        # 학습 레이블 리스트에 추가
        label_list.append(label_names[label_idx])

    # DataFrame 생성
    data_df = pd.DataFrame({'img': data_list, 'label': label_list})

    return data_df

# 부모 디렉터리 경로 지정
root_path = os.getenv('HOME') + '/aiffel/Kaggle_Hackathon/data'
# chunk file이 존재하는 경로 지정
dir_path = os.path.join(root_path, 'simple')
# 레이블 이름 파일이 존재하는 경로 지정
pkl_path = os.path.join(root_path, 'label_names.pkl')

# 파이프라인 함수 호출
data_df = data_pipeline(dir_path, pkl_path, 50)

# 만들어진 데이터프레임 확인
print(data_df.head())